# Capstone Project on Seattle car accident severity

In [2]:
import numpy as np
import pandas as pd

In [4]:
#Reading data from CSV file
df1=pd.read_csv(r'C:\\Users\\Akshay.DESKTOP-E4NEDBT\\Downloads\\Data-Collisions-road.csv')

In [5]:
df1.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [6]:
# Creating a new data frame by droping all extra coulumns
df2=df1[['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND']]
df2.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


In [7]:
# Label encoding non numeric coulumns
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df2['WEATHER_ENC'] = le.fit_transform(df2['WEATHER'].astype(str))
df2['ROADCOND_ENC'] = le.fit_transform(df2['ROADCOND'].astype(str))
df2['LIGHTCOND_ENC'] = le.fit_transform(df2['LIGHTCOND'].astype(str))

C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [8]:
#Final Data after label encoding
df2.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND,WEATHER_ENC,ROADCOND_ENC,LIGHTCOND_ENC
0,2,Overcast,Wet,Daylight,4,8,5
1,1,Raining,Wet,Dark - Street Lights On,6,8,2
2,1,Overcast,Dry,Daylight,4,0,5
3,1,Clear,Dry,Daylight,1,0,5
4,2,Raining,Wet,Daylight,6,8,5


In [9]:
#Finding counts of values of diffrent severity in data
df2['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [10]:
# Down sampling data with severity code 1
from sklearn.utils import resample
df2_major=df2[df2.SEVERITYCODE==1]
df2_minor=df2[df2.SEVERITYCODE==2]
df2_major_dsample=resample(df2_major,
                           replace= False,
                           n_samples=58188,
                           random_state=123)
balanced_df2=pd.concat([df2_major_dsample,df2_minor])
balanced_df2['SEVERITYCODE'].value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [11]:
#Defining array X
X=np.asarray(df2[['WEATHER_ENC','ROADCOND_ENC','LIGHTCOND_ENC']])
X[0:5]

array([[4, 8, 5],
       [6, 8, 2],
       [4, 0, 5],
       [1, 0, 5],
       [6, 8, 5]])

In [12]:
# Defining array y
y=np.asarray(df2['SEVERITYCODE'])
y[0:5]

array([2, 1, 1, 1, 2], dtype=int64)

In [13]:
# Normalizing the data 
from sklearn import preprocessing
X=preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 0.22981187,  1.39847224,  0.25900713],
       [ 0.87758556,  1.39847224, -1.36653782],
       [ 0.22981187, -0.73846749,  0.25900713],
       [-0.74184867, -0.73846749,  0.25900713],
       [ 0.87758556,  1.39847224,  0.25900713]])

In [14]:
# Train/Test split of data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
print('Train set:',X_train.shape,y_train.shape)
print('Test set:',X_test.shape,y_test.shape)

Train set: (136271, 3) (136271,)
Test set: (58402, 3) (58402,)


### K-Nearest Neighbour

In [15]:
# Building the KNN Model
from sklearn.neighbors import KNeighborsClassifier
k=25

In [16]:
# Train Model and predict
neigh=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
neigh
Kyhat=neigh.predict(X_test)
Kyhat[0:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [17]:
# Jaccard similarity score
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, Kyhat)

C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


0.7048217526797027

In [18]:
# F1- score
from sklearn.metrics import f1_score
f1_score(y_test, Kyhat, average='weighted') 

0.582846780130164

Model is most accurate when K=25

### Decision Tree

In [20]:
# Building the Decision tree
from sklearn.tree import DecisionTreeClassifier
colDataTree = DecisionTreeClassifier(criterion="entropy",max_depth = 7)
colDataTree
colDataTree.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
# Train model and predict
DTyhat = colDataTree.predict(X_test)
print(DTyhat [0:5])
print(y_test [0:5])

[1 1 1 1 1]
[2 1 2 2 1]


In [22]:
# Jaccard similarity score
jaccard_similarity_score(y_test,DTyhat)

C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


0.7047532618745933

In [23]:
# F1- score
f1_score(y_test,DTyhat,average='macro')

0.41357355147211866

Model is most accurate with max depth of 7

### Logistic Regression

In [24]:
# Building logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR= LogisticRegression(C=6, solver='liblinear').fit(X_train,y_train)
LR

LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
# Predict result
LRyhat = LR.predict(X_test)
LRyhat

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [26]:
yhat_prob= LR.predict_proba(X_test)
yhat_prob

array([[0.74712365, 0.25287635],
       [0.83670319, 0.16329681],
       [0.67181946, 0.32818054],
       ...,
       [0.67181946, 0.32818054],
       [0.70037211, 0.29962789],
       [0.75391483, 0.24608517]])

In [27]:
#Jaccard similarity score
jaccard_similarity_score(y_test,LRyhat)

C:\Users\Akshay.DESKTOP-E4NEDBT\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


0.7048559980822574

In [28]:
#F1-score
f1_score(y_test,LRyhat,average='macro')

0.41344019604889165

In [29]:
# LOGLOSS
yhat_prob = LR.predict_proba(X_test)
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

0.5989587236357746